In [1]:
import sys
import pandas as pd
sys.path.insert(0, '..')
from cider.featurizer import *
from helpers.utils import *

import os
del os.environ['HADOOP_CONF_DIR']

config = '../configs/config_conner.yml'

### Load from datastore

In [2]:
datastore = DataStore(config)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/connermanuel/hadoop/spark-3.2.0/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/connermanuel/hadoop/hadoop-3.3.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2021-11-18 09:19:42,177 WARN util.Utils: Your hostname, Hamish resolves to a loopback address: 127.0.1.1; using 172.25.6.121 instead (on interface eth0)
2021-11-18 09:19:42,178 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-11-18 09:19:43,753 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform..

In [3]:
featurizer = Featurizer(datastore)

Loading CDR...
Loading recharges...
Loading mobile data...
Loading mobile data...
Loading antennas...


In [4]:
featurizer.ds.cdr.count()

100000

### Deduplicate

In [17]:
cdr = pd.read_csv('../synthetic_data/cdr.csv')
recharges = pd.read_csv('../synthetic_data/recharges.csv')

assert featurizer.ds.cdr.count() == len(cdr)

featurizer.ds.cdr = featurizer.ds.cdr.union(featurizer.ds.cdr)
featurizer.ds.recharges = featurizer.ds.recharges.union(featurizer.ds.recharges)
assert featurizer.ds.cdr.count() == 2*len(cdr)
assert featurizer.ds.recharges.count() == 2*len(recharges)

featurizer.ds.deduplicate()
assert featurizer.ds.cdr.count() == len(cdr)
assert featurizer.ds.recharges.count() == len(recharges)

### Filter Dates

In [ ]:
featurizer.ds.cdr.agg({'day':'min'}).collect()[0][0]

In [ ]:
featurizer = Featurizer(datastore)

for df in [featurizer.ds.cdr, featurizer.ds.recharges, featurizer.ds.mobiledata, featurizer.ds.mobilemoney]:
    assert df.agg({'day':'min'}).collect()[0][0] == pd.to_datetime('2020-01-01')
    assert df.agg({'day':'max'}).collect()[0][0] == pd.to_datetime('2020-02-29')

featurizer.ds.filter_dates('2020-01-05', '2020-02-01')
for df in [featurizer.ds.cdr, featurizer.ds.recharges, featurizer.ds.mobiledata, featurizer.ds.mobilemoney]:
    assert df.agg({'day':'min'}).collect()[0][0] == pd.to_datetime('2020-01-05')
    assert df.agg({'day':'max'}).collect()[0][0] == pd.to_datetime('2020-02-01')

### Remove Spammers

In [ ]:
# Test for number of spammers identified

for spammer_threshold in [0, 1.5, 2]:
    
    featurizer = Featurizer(datastore)

    spammers = len(featurizer.ds.remove_spammers(spammer_threshold=spammer_threshold))
    
    cdr = pd.read_csv('../synthetic_data/cdr.csv')
    cdr['timestamp'] = pd.to_datetime(cdr['timestamp'])
    cdr['day'] = cdr['timestamp'].dt.date
    grouped = cdr.groupby(['caller_id', 'txn_type']).agg({'day':['count', 'nunique']}).reset_index()
    assert len(grouped[grouped['day']['count'] > spammer_threshold*grouped['day']['nunique']]['caller_id'].unique()) == spammers

In [ ]:
# Test for removal of spammers

spammer_threshold = 1

featurizer = Featurizer(config)

spammers = featurizer.remove_spammers(spammer_threshold=spammer_threshold)

for df in [featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]:
    
    caller_ids = [item[0] for item in featurizer.cdr.select('caller_id').collect()]
    assert set(spammers).intersection(set(caller_ids)) == set()
    
    if 'recipient_id' in df.columns:
        recipient_ids = [item[0] for item in featurizer.cdr.select('caller_id').collect()]
        assert set(spammers).intersection(set(recipient_ids)) == set()

### Filter Outlier Days

In [ ]:
# Test for number of days identified

for num_sds in [1, 1.5, 2]:
    featurizer = Featurizer(config)

    outliers = [pd.to_datetime(item).tz_localize(None) for item in featurizer.filter_outlier_days(num_sds)]

    cdr = pd.read_csv('../synthetic_data/cdr.csv')
    cdr['timestamp'] = pd.to_datetime(cdr['timestamp'])
    cdr['day'] = cdr['timestamp'].dt.floor('d')
    grouped = cdr.groupby('day', as_index=False).agg('count')
    u = grouped['txn_type'].mean() + num_sds*grouped['txn_type'].std()
    l = grouped['txn_type'].mean() - num_sds*grouped['txn_type'].std()
    outliers_manual = grouped[(grouped['txn_type'] < l) | (grouped['txn_type'] > u)]['day'].astype('object')
    assert list(outliers_manual.unique()) == outliers

In [ ]:
# Test for removal of days

num_sds = 2

featurizer = Featurizer(config)

original_counts = []
for df in [featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]:
    original_counts.append(df.select('day').distinct().count())
    
num_outliers = len(featurizer.filter_outlier_days(num_sds))

for d, df in enumerate([featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]):
    assert(df.select('day').distinct().count() == original_counts[d] - num_outliers)

### Diagnostic Statistics

In [ ]:
featurizer = Featurizer(config)

statistics = featurizer.diagnostic_statistics('test_output')

for fname, name in [('cdr', 'CDR')]:
    
    df = pd.read_csv('../synthetic_data/' + fname + '.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    assert((df['timestamp'].max() - df['timestamp'].min()).days + 1 == statistics[name]['Days'])
    assert(len(df) == statistics[name]['Transactions'])
    assert(len(df['caller_id'].unique()) == statistics[name]['Subscribers'])
    if 'recipient_id' in df.columns:
        assert(len(df['recipient_id'].unique()) == statistics[name]['Recipients'])

### Diagnostic Plots

In [ ]:
featurizer = Featurizer(config)

featurizer.diagnostic_plots('test_output')

### CDR Features

In [5]:
# featurizer = Featurizer(datastore)
featurizer.location_features()

Calculating spatial features...


/home/connermanuel/envs/poetry/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/mnt/d/College/Fall 2021/URAP/cider/notebooks/../cider/featurizer.py:325: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  antennas = gpd.sjoin(antennas, shapefile, op='within', how='left').drop('index_right', axis=1)
/mnt/d/College/Fall 2021/URAP/cider/notebooks/../cider/featurizer.py:325: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` 

outfname: file:///mnt/d/College/Fall 2021/URAP/cider/outputs/featurizer//datasets/countbyregions.csv
outfolder: file:///mnt/d/College/Fall 2021/URAP/cider/outputs/featurizer//datasets/countbyregions
Trying to write csv...


Wrote csv!
outfname: file:///mnt/d/College/Fall 2021/URAP/cider/outputs/featurizer//datasets/countbyprefectures.csv
outfolder: file:///mnt/d/College/Fall 2021/URAP/cider/outputs/featurizer//datasets/countbyprefectures
Trying to write csv...
Wrote csv!
outfname: file:///mnt/d/College/Fall 2021/URAP/cider/outputs/featurizer//datasets/uniqueregions.csv
outfolder: file:///mnt/d/College/Fall 2021/URAP/cider/outputs/featurizer//datasets/uniqueregions
Trying to write csv...


Wrote csv!


In [6]:
featurizer.cdr_features()

Calculating CDR features...
outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/subscribers.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/subscribers
Trying to write csv...
Wrote csv!
old_outputs = file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer/
Making /mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_features/0to1000


mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=dsBHAdXrrk/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=JGPCbfDGes/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=dYwshzRseD/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=ygMEXUQDbn/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=YtvkGlMWwe/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=ZuLaiYuunH/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//dataset

mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=kNflXMaRCt/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=CGbKIWfKzS/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=XiKpNMsHxn/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=NfhDWwJCLO/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=aNGoLSaKjJ/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=LySPboXRJq/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//dataset

mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=kHvRKmzbkc/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=ruNGTTGsry/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=dWgAxvDvhT/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=TTukgLnomN/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=isjaYQNfQZ/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=lpAchcvhBO/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//dataset

mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=YhCoydLtxG/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=wmyAJISQoE/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=uqloFKfVPJ/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=eGwidYFuZD/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=FqzmWqeIBN/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_records/0to1000/name=CXRpRFmPNR/*.csv': No such file or directory
mv: cannot stat '/mnt/d/College/f2021/URAP/cider/outputs/featurizer//dataset

...

         No antennas file was given and records are using antennas for position.
[x] 206 records from 2020-01-01 04:01:48 to 2020-02-29 07:38:55
[x] 182 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 223 records from 2020-01-01 00:53:36 to 2020-02-29 17:27:34
[x] 196 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 199 records from 2020-01-01 08:21:41 to 2020-02-29 21:49:26
[x] 181 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 195 records from 2020-01-01 04:37:45 to 2020-02-29 23:53:08
[x] 177 cont

[x] 194 records from 2020-01-01 00:10:35 to 2020-02-29 23:33:15
[x] 179 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 193 records from 2020-01-01 06:33:55 to 2020-02-29 20:31:44
[x] 174 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 211 records from 2020-01-01 10:27:25 to 2020-02-29 15:28:39
[x] 188 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 183 records from 2020-01-01 15:00:23 to 2020-02-29 19:32:20
[x] 170 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home


         No antennas file was given and records are using antennas for position.
[x] 183 records from 2020-01-01 00:10:34 to 2020-02-29 15:22:18
[x] 169 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 224 records from 2020-01-01 02:47:26 to 2020-02-29 21:19:51
[x] 206 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 216 records from 2020-01-01 06:57:41 to 2020-02-29 19:33:39
[x] 195 contacts
[ ] No attribute stored
[ ] No antenna stored
[ ] No recharges
[x] Has home
[x] Has texts
[x] Has calls
[ ] No network
         No antennas file was given and records are using antennas for position.
[x] 204 records from 2020-01-01 08:37:16 to 2020-02-29 16:49:58
[x] 179 cont

outputs = file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer/
outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_features/all.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/bandicoot_features/all
Trying to write csv...


Wrote csv!


In [5]:
featurizer.international_features()

Calculating international features...
outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/internatonal_transactions.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/internatonal_transactions
Trying to write csv...
Wrote csv!


In [6]:
featurizer.recharges_features()

Calculating recharges features...
outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/recharges_feats.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/recharges_feats
Trying to write csv...
Wrote csv!


In [7]:
featurizer.mobiledata_features()

Calculating mobile data features...
outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/mobiledata_features.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/mobiledata_features
Trying to write csv...
Wrote csv!


In [8]:
featurizer.mobilemoney_features()

Calculating mobile money features...


outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/mobilemoney_feats.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/mobilemoney_feats
Trying to write csv...


Wrote csv!


In [9]:
featurizer.all_features()

outfname: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/features.csv
outfolder: file:///mnt/d/College/f2021/URAP/cider/outputs/featurizer//datasets/features
Trying to write csv...


Wrote csv!


In [ ]:
# Total number of records
cdr = pd.read_csv('../synthetic_data/cdr.csv')
outgoing = cdr.groupby('caller_id').agg('count')\
    [['txn_type']]
incoming = cdr.groupby('recipient_id').agg('count')\
    [['txn_type']]
txns = pd.concat([outgoing, incoming])
txns['caller_id'] = txns.index
txn_count = txns.groupby('caller_id').agg('sum')
txn_count['name'] = txn_count.index
feats = featurizer.features['cdr'].toPandas()\
    [['name', 'cdr_reporting__number_of_records']]
feats['cdr_reporting__number_of_records'] = feats['cdr_reporting__number_of_records']\
    .astype('float')
merged = txn_count.merge(feats, on='name', how='outer')
assert len(merged[merged['txn_type'] != merged['cdr_reporting__number_of_records']]) == 0

In [ ]:
# Mean call time
cdr = pd.read_csv('../synthetic_data/cdr.csv')
txns = pd.concat([cdr[['caller_id', 'duration']].rename({'caller_id':'name'}, axis=1), 
            cdr[['recipient_id', 'duration']].rename({'recipient_id':'name'}, axis=1)])
txns = txns.groupby('name', as_index=False).agg('mean')
feats = featurizer.features['cdr'].toPandas()
feats['cdr_call_duration__allweek__allday__call__mean'] = \
    feats['cdr_call_duration__allweek__allday__call__mean'].astype('float')
merged = feats[['name', 'cdr_call_duration__allweek__allday__call__mean']]\
    .merge(txns, on='name')
assert len(merged[merged['duration'].astype('int') != \
       merged['cdr_call_duration__allweek__allday__call__mean'].astype('int')]) == 0

In [ ]:
# Number of contacts
cdr = pd.read_csv('../synthetic_data/cdr.csv')
txns = cdr[cdr['txn_type'] == 'call']
txns = pd.concat([txns[['caller_id', 'recipient_id', 'duration']]\
                      .rename({'caller_id':'name', 'recipient_id':'contact'}, axis=1), 
                  txns[['caller_id','recipient_id', 'duration']]\
                      .rename({'recipient_id':'name', 'caller_id':'contact'}, axis=1)])
contacts = pd.DataFrame(txns.groupby('name')['contact'].nunique())
feats = featurizer.features['cdr'].toPandas()\
    [['name', 'cdr_number_of_contacts__allweek__allday__call']]
feats['cdr_number_of_contacts__allweek__allday__call'] = \
    feats['cdr_number_of_contacts__allweek__allday__call'].astype('int')
merged = feats.merge(contacts, on='name', how='outer')
assert len(merged[merged['contact'] != \
                  merged['cdr_number_of_contacts__allweek__allday__call']]) == 0

### International Features

In [ ]:
# Number of outgoing transactions
cdr = pd.read_csv('../synthetic_data/cdr.csv')
cdr = cdr[cdr['international'] == 'international']
feats = featurizer.features['international'].toPandas()
inter = cdr.groupby('caller_id', as_index=False).agg('count')[['caller_id', 'txn_type']]\
    .rename({'caller_id':'name'}, axis=1)
merged = feats.merge(inter, on='name')
assert list(merged['international_all__recipient_id__count'].astype('int')) == \
    list(merged['txn_type'].astype('int'))

In [ ]:
# Total call duration
cdr = pd.read_csv('../synthetic_data/cdr.csv')
cdr = cdr[cdr['international'] == 'international']
feats = featurizer.features['international'].toPandas()
inter = cdr.groupby('caller_id', as_index=False).agg('sum')[['caller_id', 'duration']]\
    .rename({'caller_id':'name'}, axis=1)
merged = feats.merge(inter, on='name')
assert list(merged['international_call__duration__sum'].fillna(0).astype('float')) == \
    list(merged['duration'].astype('float'))

### Mobile Data Features

In [ ]:
# Number of unique days
feats = featurizer.features['mobiledata'].toPandas()
data = pd.read_csv('../synthetic_data/mobiledata.csv')
data['day'] = data['timestamp'].apply(lambda x: x[:10])
days = pd.DataFrame(data.groupby('caller_id')['day'].nunique())
days['name'] = days.index
merged = days.merge(feats, on='name')
assert len(merged[merged['day'] != merged['mobiledata_num_days']]) == 0

In [ ]:
# Max volume
feats = featurizer.features['mobiledata'].toPandas()
data = pd.read_csv('../synthetic_data/mobiledata.csv')
data = data.groupby('caller_id', as_index=False).agg('max')\
    .rename({'caller_id':'name'}, axis=1)
merged = data.merge(feats, on='name')
assert list(merged['volume'].astype('int')) == \
    list(merged['mobiledata_max_volume'].astype('int'))

### Mobile Money Features

In [ ]:
# Minimum outgoing p2p volume
feats = featurizer.features['mobilemoney'].toPandas()
mm = pd.read_csv('../synthetic_data/mobilemoney.csv')
mm = mm[mm['txn_type'] == 'p2p']
mm = mm.groupby('caller_id', as_index=False).agg('min')\
    .rename({'caller_id':'name'}, axis=1)
merged = mm.merge(feats, on='name', how='outer').fillna(0)
assert (merged['mobilemoney_outgoing_p2p_amount_min'].astype('float').round(1) == \
    merged['amount'].astype('float').round(1)).all()

In [ ]:
# Minimum outgoing p2p volume
feats = featurizer.features['mobilemoney'].toPandas()
mm = pd.read_csv('../synthetic_data/mobilemoney.csv')
outgoing = mm[['caller_id', 'sender_balance_before']]\
    .rename({'caller_id':'name', 'sender_balance_before':'balance'}, axis=1)
incoming = mm[['recipient_id', 'recipient_balance_before']]\
    .rename({'recipient_id':'name', 'recipient_balance_before':'balance'}, axis=1)
mm = pd.concat([outgoing, incoming])
mm = mm.groupby('name', as_index=False).agg('mean')
merged = mm.merge(feats, on='name', how='outer').fillna(0)
assert (merged['mobilemoney_all_all_balance_before_mean'].astype('float').round(1) == \
    merged['balance'].astype('float').round(1)).all()

### Recharges Features

In [ ]:
# Total recharge
feats = featurizer.features['recharges'].toPandas()
recharges = pd.read_csv('../synthetic_data/recharges.csv')\
    .rename({'caller_id':'name'}, axis=1)
recharges = recharges.groupby('name', as_index=False).agg('sum')
merged = recharges.merge(feats, on='name')
assert (merged['amount'].astype('float') == merged['recharges_sum'].astype('float')).all()

### Location Features

In [ ]:
feats = featurizer.features['location'].toPandas()
cdr = pd.read_csv('../synthetic_data/cdr.csv')
antennas = pd.read_csv('../synthetic_data/antennas.csv')
antennas = gpd.GeoDataFrame(antennas, geometry=gpd.points_from_xy(antennas['longitude'], antennas['latitude']))
antennas.crs = {"init":"epsg:4326"}
prefectures = gpd.read_file('../synthetic_data/prefectures.geojson')
antennas = gpd.sjoin(antennas, prefectures, op='within', how='left')[['antenna_id', 'region']]
antennas['region'] = antennas['region'].fillna('Unknown')
outgoing = cdr[['caller_id', 'caller_antenna']]\
    .rename({'caller_id':'name', 'caller_antenna':'antenna_id'}, axis=1)\
    .merge(antennas, on='antenna_id', how='left')
incoming = cdr[['recipient_id', 'recipient_antenna']]\
    .rename({'recipient_id':'name', 'recipient_antenna':'antenna_id'}, axis=1)\
    .merge(antennas, on='antenna_id', how='left')
cdr = pd.concat([outgoing, incoming])
cdr['region'] = cdr['region'].fillna('Unknown')

In [ ]:
# Unique prefectures
unique = pd.DataFrame(cdr.groupby('name')['region'].nunique())
merged = unique.merge(feats[['name', 'location_count(prefectures)']], on='name')
assert (merged['region'].astype('int') == merged['location_count(prefectures)'].astype('int')).all()

In [ ]:
# Count per prefecture
cdr['call'] = 1
counts = cdr.groupby(['name', 'region'], as_index=False).count()\
    .rename({'call':'count'}, axis=1)\
    .pivot(index='name', columns='region', values='count').fillna(0)
merged = counts.merge(feats, on='name', how='inner')
for prefecture in counts.columns:
    assert (merged[prefecture].astype('float') == merged['location_prefectures_' + prefecture].astype('float'))\
        .all(), prefecture